In [18]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import html
from pytorch_pretrained_bert import BertTokenizer


In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

11/30/2018 00:44:54 - INFO - pytorch_pretrained_bert.tokenization -   loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/joeyli/.pytorch_pretrained_bert/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [45]:
path = os.path.join('data', 'tweets.csv')
df = pd.read_csv(path, encoding='latin1', usecols=[0, 5], header=None, names=['tag', 'tweet'])

In [194]:
def valid_word(word):
    if word.startswith('@'):
        return False
    try:
        word.encode('ascii')
    except UnicodeEncodeError:
        return False
    
    return True

def clean_tweet(tweet):
    tweet = html.unescape(tweet)
    words = tweet.split()
    words = [word.lower() for word in words if valid_word(word)]
    
    return ' '.join(words)

def check_invalid(tweet):
    tweet = html.unescape(tweet)
    words = tweet.split()
    invalid_count = 0
    for word in words:
        try:
            word.encode('ascii')
        except UnicodeEncodeError:
            return 1
    return 0

In [195]:
df['cleaned_tweet'] = df.tweet.apply(clean_tweet)

In [196]:
df['invalid'] = df.tweet.apply(check_invalid)

In [217]:
cleandf = df[df.invalid == 0]

In [218]:
cleandf = cleandf.reset_index(drop=True)

In [219]:
train, test = train_test_split(cleandf, test_size=0.1)

In [220]:
train = train.reset_index(drop=True)

In [221]:
test = test.reset_index(drop=True)

In [223]:
train.to_csv('data/train.csv', index=False, columns=['tag', 'cleaned_tweet'])

In [224]:
test.to_csv('data/test.csv', index=False, columns=['tag', 'cleaned_tweet'])

In [20]:
train = pd.read_csv('data/train.csv', dtype={'tag':int, 'cleaned_tweet':str})

In [12]:
def tweet_len(tweet):
    return len(str(tweet).split())

In [23]:
def tokenize(tweet):
    return ' '.joint(tokenizer.tokenize(str(tweet)))

In [25]:
train['tokens'] = train.cleaned_tweet.apply(tokenize)

In [26]:
train.to_csv('data/train.csv', index=False, columns=['tag', 'tokens'])

In [27]:
test = pd.read_csv('data/test.csv', dtype={'tag':int, 'cleaned_tweet':str})
test['tokens'] = test.cleaned_tweet.apply(tokenize)
test.to_csv('data/test.csv', index=False, columns=['tag', 'tokens'])

In [31]:
tokenizer.tokenize(' s\ns ')

['s', 's']